# This source code searches for the ideal number of mixed layers on Inception V3 with the Cropped-PlantDoc dataset

This raw file supports the paper [Color-aware two-branch DCNN for efficient plant disease classification](https://github.com/joaopauloschuler/two-branch-plant-disease).

You might need to install this on your system:

apt-get install python3-opencv git

In [ ]:
import os

if not os.path.isdir('k'):
 !git clone https://github.com/joaopauloschuler/k-neural-api.git k
else:
 !cd k && git pull

!cd k && pip install .

In [ ]:
import cai.layers
import cai.datasets
import cai.models
import cai.densenet
import numpy as np
from tensorflow import keras
from tensorflow.keras import backend
from tensorflow.keras import layers
import gc
import multiprocessing
import random
import tensorflow as tf
print("Tensorflow version:", tf.version.VERSION)
print("Keras version:", keras.__version__)
import PIL
import cv2
#print("Pillow version:", PIL.PILLOW_VERSION)
from sklearn.metrics import classification_report

Tensorflow version: 2.4.1
Keras version: 2.4.0


In [ ]:
if not os.path.isdir('up'):
  !git clone https://github.com/joaopauloschuler/PlantDoc-Object-Detection-Dataset.git up

In [ ]:
up_data_dir = os.getcwd()+"/up/TRAIN"
up_test_dir = os.getcwd()+"/up/TEST"
data_dir = "cropped_train/"
test_dir = "cropped_test/"
all_dir = "cropped_all/"

In [ ]:
def CropImages(orig_data_dir, dest_data_dir, csv_file, include_original=False, verbose=True, prefix='img_'):
  # Load the CSV file.
  with open (csv_file, "r") as myfile:
    csv_lines=myfile.readlines()

  # Transform the CSV file into an array  
  line_count = 0
  a_lines = []
  for str_line in csv_lines:
    if line_count > 0: 
      a_line = str_line.replace('\n', '').split(',')
      a_lines.append(a_line)
    line_count = line_count + 1
  #a_lines = np.array(a_lines, dtype=object)
  
  # create destination folder
  if not os.path.isdir(dest_data_dir):
    os.mkdir(dest_data_dir)

  # Save cropped images
  line_count = 0
  failed_count = 0
  original_count = 0
  last_image_file = ''
  for a_line in a_lines:
    dest_folder_name = dest_data_dir+'/'+a_line[3]
    dest_file_name = dest_folder_name+'/'+prefix+str(line_count)+'.jpg'
    orig_file_name = orig_data_dir+'/'+ a_line[0]
    print(line_count,':', a_line, dest_file_name)
    if last_image_file != orig_file_name:
      if os.path.isfile(orig_file_name):
        img = cv2.imread(orig_file_name)
        #cv2_imshow(img)
        last_image_file = orig_file_name
        can_load = True
        original_count = original_count + 1
        if verbose: print("Original imgage shape",img.shape)
        if (include_original):
          if not os.path.isdir(dest_folder_name):
            os.mkdir(dest_folder_name)
          cv2.imwrite(dest_folder_name+'/ori_'+prefix+str(original_count)+'.jpg',img)  
      else:
        if verbose: print('File does not exist:', orig_file_name)
        can_load = False
        #break
    if (can_load):
      y1 = int(a_line[4])
      x1 = int(a_line[5])
      y2 = int(a_line[6])
      x2 = int(a_line[7])

      x_min = np.min([x1, x2])
      x_max = np.max([x1, x2])
      y_min = np.min([y1, y2])
      y_max = np.max([y1, y2])

      crop_img = img[x_min:x_max, y_min:y_max]
      if verbose: print("Cropped imgage shape",crop_img.shape,':', x_min, x_max, y_min, y_max,' deltas:',x_max - x_min, y_max - y_min)
      if not os.path.isdir(dest_folder_name):
        os.mkdir(dest_folder_name)
      if ( (crop_img.shape[0]==0) or (crop_img.shape[1]==0)):
        failed_count = failed_count + 1
        if verbose: print("Failed cropping.");
      else:
        cv2.imwrite(dest_file_name,crop_img)
    else:
      failed_count = failed_count + 1
    line_count = line_count + 1
  print("Processed files:", line_count, "Failed count: ", failed_count, "Original count:", original_count)

In [ ]:
if not os.path.isdir(data_dir):
  CropImages(orig_data_dir=up_data_dir, dest_data_dir=data_dir, csv_file='up/train_labels.csv', include_original=False, prefix='train_')

In [ ]:
if not os.path.isdir(test_dir):
  CropImages(orig_data_dir=up_test_dir, dest_data_dir=test_dir, csv_file='up/test_labels.csv', include_original=False, prefix='test_')

In [ ]:
print(os.listdir(data_dir))
print(os.listdir(test_dir))
if not os.path.isdir('cropped_test/Potato leaf'):
  !mkdir 'cropped_test/Potato leaf'
if not os.path.isdir('cropped_test/Tomato two spotted spider mites leaf'):
  !mkdir 'cropped_test/Tomato two spotted spider mites leaf'

['grape leaf black rot', 'Apple Scab Leaf', 'Tomato Septoria leaf spot', 'Peach leaf', 'Tomato leaf late blight', 'Potato leaf late blight', 'Potato leaf early blight', 'Tomato leaf bacterial spot', 'Corn Gray leaf spot', 'Bell_pepper leaf', 'Apple rust leaf', 'Strawberry leaf', 'grape leaf', 'Cherry leaf', 'Corn rust leaf', 'Tomato leaf yellow virus', 'Tomato mold leaf', 'Potato leaf', 'Bell_pepper leaf spot', 'Squash Powdery mildew leaf', 'Apple leaf', 'Corn leaf blight', 'Raspberry leaf', 'Blueberry leaf', 'Tomato leaf', 'Soyabean leaf', 'Tomato two spotted spider mites leaf', 'Tomato Early blight leaf', 'Tomato leaf mosaic virus']
['grape leaf black rot', 'Apple Scab Leaf', 'Tomato Septoria leaf spot', 'Peach leaf', 'Tomato leaf late blight', 'Potato leaf late blight', 'Potato leaf early blight', 'Tomato leaf bacterial spot', 'Corn Gray leaf spot', 'Bell_pepper leaf', 'Apple rust leaf', 'Strawberry leaf', 'grape leaf', 'Cherry leaf', 'Corn rust leaf', 'Tomato leaf yellow virus', 'T

In [ ]:
if not os.path.isdir(all_dir):
  !mkdir cropped_all
  ! cp -r cropped_test/* cropped_all/
  ! cp -r cropped_train/* cropped_all/

In [ ]:
datagen = cai.util.create_image_generator(vertical_flip=True, rotation_range=90) # width_shift_range=0.5, height_shift_range=0.5

def lrscheduler(epoch):
  return 0.01 * (0.99**epoch)

# Load the dataset again with LAB

In [ ]:
num_classes = 28
batch_size = 32
epochs = 240
target_size_x = 224 # default value is 224
target_size_y = 224 # default value is 224

In [ ]:
train_x, val_x, test_x, train_y, val_y, test_y, classweight, classes = cai.datasets.load_images_from_folders(seed=7,
  root_dir=all_dir, lab=True, 
  verbose=True, bipolar=False, base_model_name='plant_doc',
  has_training=True, has_validation=True, has_testing=True,
  training_size=0.65, validation_size=0.15, test_size=0.2,
  target_size=(target_size_x, target_size_y))

print(train_x.shape,val_x.shape,test_x.shape)
print(train_y.shape,val_y.shape,test_y.shape)

Loading  29  classes.
loading train images
train shape is: (5761, 224, 224, 3)
loading validation images
validation shape is: (1333, 224, 224, 3)
loading test images
test shape is: (1789, 224, 224, 3)
Converting RGB to LAB: 
Converting training.
1000  images converted to lab.
2000  images converted to lab.
3000  images converted to lab.
4000  images converted to lab.
5000  images converted to lab.
Converting validation.
1000  images converted to lab.
Converting test.
1000  images converted to lab.
Channel  0  min: 0.0  max: 1.0
Channel  1  min: 0.11655408  max: 0.9557502
Channel  2  min: 1.0877848e-05  max: 0.9704958
Loaded.
(5761, 224, 224, 3) (1333, 224, 224, 3) (1789, 224, 224, 3)
(5761, 29) (1333, 29) (1789, 29)


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:72: FutureWarning: Pass classes=[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28], y=[ 0  0  0 ... 28 28 28] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


In [ ]:
def work_on_inception_v3(show_model=False, run_fit=False, test_results=False, calc_f1=False):
  monitor='val_accuracy'
  if (calc_f1): 
    test_results=True
  if (show_model):
    input_shape = (target_size_x, target_size_y, 3)
  else:
    input_shape = (None, None, 3)
  deep_two_paths_compression=0.6
  for l_ratio in [0.2]:
    for max_mix_idx in [0, 1, 3, 5, 7, 9, 10]:
      if l_ratio < 0:
          two_paths_first_block=False
          two_paths_second_block=False
          deep_two_paths=False
      else:
          two_paths_first_block=True
          two_paths_second_block=False
          deep_two_paths=False
      basefilename = 'JP27B17-InceptionV3-CroppedPlantDoc-'+str(l_ratio)
      best_result_file_name = basefilename+'-best_result.hdf5'
      print('Running: '+basefilename)
      model = cai.models.compiled_two_path_inception_v3(
        input_shape=input_shape,
        classes=29, 
        two_paths_first_block=two_paths_first_block,
        two_paths_second_block=two_paths_second_block,
        deep_two_paths=deep_two_paths,
        deep_two_paths_compression=deep_two_paths_compression,
        l_ratio=l_ratio,
        ab_ratio=(1-l_ratio),
        max_mix_idx=max_mix_idx, 
        model_name='two_path_inception_v3_mixed5'
        )

      if (show_model): model.summary()

      save_best = keras.callbacks.ModelCheckpoint(
            filepath=best_result_file_name,
            monitor=monitor,
            verbose=1,
            save_best_only=True,
            save_weights_only=False,
            mode='max',
            save_freq='epoch')

      if (run_fit):
        history = model.fit(
          x = datagen.flow(train_x, train_y, batch_size=batch_size),
          epochs=epochs,
          batch_size=batch_size,
          validation_data=(val_x, val_y),
          callbacks=[save_best, tf.keras.callbacks.LearningRateScheduler(lrscheduler)], # cai.densenet.cyclical_smooth_lrscheduler
          workers=max([multiprocessing.cpu_count(), 4]) # this option 
          )
      if (test_results):
        print('Best Model Results: '+basefilename)
        model = keras.models.load_model(best_result_file_name, custom_objects={'CopyChannels': cai.layers.CopyChannels})
        evaluated = model.evaluate(test_x,test_y)
        for metric, name in zip(evaluated,["loss","acc"]):
              print(name,metric)
      if (calc_f1):
        pred_y = model.predict(test_x)
        print("Predicted Shape:", pred_y.shape)
        pred_classes_y = np.array(list(np.argmax(pred_y, axis=1)))
        test_classes_y = np.array(list(np.argmax(test_y, axis=1)))
        print("Pred classes shape:",pred_classes_y.shape)
        print("Test classes shape:",test_classes_y.shape)
        report = classification_report(test_classes_y, pred_classes_y, digits=4)
        print(report)
      print('Finished: '+basefilename)

# Show Models

In [ ]:
work_on_inception_v3(show_model=True, run_fit=False, test_results=False)

Running: JP27B17-InceptionV3-CroppedPlantDoc-0.2
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
copy_channels (CopyChannels)    (None, 224, 224, 1)  0           input_1[0][0]                    
__________________________________________________________________________________________________
copy_channels_1 (CopyChannels)  (None, 224, 224, 2)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 6)  54          copy_channels[0][0]              
_____________________________________________

# Fitting

In [ ]:
work_on_inception_v3(show_model=False, run_fit=True, test_results=True)

Running: JP27B17-InceptionV3-CroppedPlantDoc-0.2
Epoch 1/240
181/181 [==============================] - 24s 108ms/step - loss: 2.9412 - accuracy: 0.1770 - val_loss: 3.4970 - val_accuracy: 0.0720

Epoch 00001: val_accuracy improved from -inf to 0.07202, saving model to JP27B17-InceptionV3-CroppedPlantDoc-0.2-best_result.hdf5
Epoch 2/240
181/181 [==============================] - 17s 92ms/step - loss: 2.5021 - accuracy: 0.2659 - val_loss: 2.9790 - val_accuracy: 0.1508

Epoch 00002: val_accuracy improved from 0.07202 to 0.15079, saving model to JP27B17-InceptionV3-CroppedPlantDoc-0.2-best_result.hdf5
Epoch 3/240
181/181 [==============================] - 17s 92ms/step - loss: 2.3392 - accuracy: 0.3064 - val_loss: 3.9254 - val_accuracy: 0.1260

Epoch 00003: val_accuracy did not improve from 0.15079
Epoch 4/240
181/181 [==============================] - 17s 91ms/step - loss: 2.2031 - accuracy: 0.3396 - val_loss: 2.4025 - val_accuracy: 0.2866

Epoch 00004: val_accuracy improved from 0.15079 